# Entity Embedding for cat in mixed df:

## Import:

In [0]:
import numpy as np
import pandas as pd
from utils import *

from sklearn.model_selection import KFold
from sklearn import metrics
import tensorflow as tf
from keras import layers, optimizers, callbacks, utils
from keras.models import Model, load_model
from keras import backend as K

In [101]:
train = pd.read_csv('/content/train.csv').pipe(reduce_mem_usage)
test  = pd.read_csv('/content/test.csv').pipe(reduce_mem_usage)

print(train.shape, test.shape)

~> Memory usage of dataframe is 0.902 MG
~> Memory usage after optimization is: 0.572 MG
~> Decreased by 36.6%
------------------------------------------------------------
~> Memory usage of dataframe is 0.891 MG
~> Memory usage after optimization is: 0.568 MG
~> Decreased by 36.2%
------------------------------------------------------------
(1460, 81) (1459, 80)


In [0]:
display(train, test)

In [77]:
summary(train)

~> Dataframe has  1,460  Rows, and  81  Columns.
~> Dataframe has  19  Columns have [Missing] Values.
------------------------------------------------------------
~> There are  12 	 Columns that have [Type] =  int8 
~> There are  1 	 Columns that have [Type] =  uint8 
~> There are  12 	 Columns that have [Type] =  int8 
~> There are  20 	 Columns that have [Type] =  int16 
~> There are  3 	 Columns that have [Type] =  float16 
~> There are  20 	 Columns that have [Type] =  int16 
~> There are  2 	 Columns that have [Type] =  int32 
~> There are  3 	 Columns that have [Type] =  float16 
~> There are  2 	 Columns that have [Type] =  int32 
~> There are  43 	 Columns that have [Type] =  object 
------------------------------------------------------------


,Name,dtypes,Uniques,Missing,% Missing,has_Infinite,has_Negative,top_Frequent,% frequent
0,Id,int16,1460,0,0,False,False,1460,0.07%
1,MSSubClass,uint8,15,0,0,False,False,20,36.71%
2,MSZoning,object,5,0,0,False,False,RL,78.84%
3,LotFrontage,float16,110,259,17.74,False,False,60,11.91%
4,LotArea,int32,1073,0,0,False,False,7200,1.71%
5,Street,object,2,0,0,False,False,Pave,99.59%
6,Alley,object,2,1369,93.77,False,False,Grvl,54.95%
7,LotShape,object,4,0,0,False,False,Reg,63.36%
8,LandContour,object,4,0,0,False,False,Lvl,89.79%
9,Utilities,object,2,0,0,False,False,AllPub,99.93%


## Select Cat & Num columns:

In [0]:
cat_cols   = [col for col in train if train[col].dtype == 'O']
num_cols   = list(set(train.select_dtypes('number')) - {'SalePrice', 'Id'})
target_col = 'SalePrice'

In [103]:
len(cat_cols) + len(num_cols)

79

## Label Encoder & Filling NaNs & Normalizing:

In [107]:
# Label Encode the cat cols.
from sklearn import preprocessing

for col in cat_cols:
    le = preprocessing.LabelEncoder()
    train[col] = le.fit_transform(train[col].fillna("-1").astype(str).values)
    test[col]  = le.fit_transform(test[col].fillna("-1").astype(str).values)


scaler          = preprocessing.MinMaxScaler()
train[num_cols] = scaler.fit_transform(train[num_cols].fillna(0).values)
test[num_cols]  = scaler.transform(test[num_cols].fillna(0).values)

scaler = preprocessing.MinMaxScaler()
train[target_col] = scaler.fit_transform(train[target_col].values.reshape((-1, 1)))

print(train.isna().sum().sum(), test.isna().sum().sum())

display(train, test)

0 0
 train  Table


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,0.235294,3,0.207668,0.033420,1,0,3,3,0,4,0,5,2,2,0,5,0.666667,0.500,0.949275,0.883333,1,1,14,3,2,0.12250,2,4,2,3,4,4,3,0.125089,6,0.0,0.064212,0.140098,1,...,1,5,0.119780,0.413559,0.0,0.259231,0.333333,0.0,0.666667,0.5,0.375,0.333333,2,0.500000,6,0.000000,0,2,0.996517,2,0.50,0.386460,5,5,2,0.000000,0.111517,0.000000,0.0,0.0,0.0,0,0,0,0.0,0.090909,0.50,8,4,0.241078
1,2,0.000000,3,0.255591,0.038795,1,0,3,3,0,2,0,24,1,2,0,2,0.555556,0.875,0.753623,0.433333,1,1,10,12,3,0.00000,3,4,1,3,4,2,1,0.173281,6,0.0,0.121575,0.206547,1,...,1,5,0.212942,0.000000,0.0,0.174830,0.000000,0.5,0.666667,0.0,0.375,0.333333,3,0.333333,6,0.333333,5,2,0.983085,2,0.50,0.324401,5,5,2,0.347725,0.000000,0.000000,0.0,0.0,0.0,0,0,0,0.0,0.363636,0.25,8,4,0.203583
2,3,0.235294,3,0.217252,0.046507,1,0,0,3,0,4,0,5,2,2,0,5,0.666667,0.500,0.934783,0.866667,1,1,14,3,2,0.10125,2,4,2,3,4,3,3,0.086109,6,0.0,0.185788,0.150573,1,...,1,5,0.134465,0.419370,0.0,0.273549,0.333333,0.0,0.666667,0.5,0.375,0.333333,2,0.333333,6,0.333333,5,2,0.995522,2,0.50,0.428773,5,5,2,0.000000,0.076782,0.000000,0.0,0.0,0.0,0,0,0,0.0,0.727273,0.50,8,4,0.261908
3,4,0.294118,3,0.191693,0.038561,1,0,0,3,0,0,0,6,2,2,0,5,0.666667,0.500,0.311594,0.333333,1,1,2,5,3,0.00000,3,4,0,4,2,4,1,0.038271,6,0.0,0.231164,0.123732,1,...,1,5,0.143873,0.366102,0.0,0.260550,0.333333,0.0,0.333333,0.0,0.375,0.333333,2,0.416667,6,0.333333,3,6,0.994030,3,0.75,0.452750,5,5,2,0.000000,0.063985,0.492754,0.0,0.0,0.0,0,0,0,0.0,0.090909,0.00,8,0,0.145952
4,5,0.235294,3,0.268371,0.060576,1,0,0,3,0,2,0,15,2,2,0,5,0.777778,0.500,0.927536,0.833333,1,1,14,3,2,0.21875,2,4,2,3,4,1,3,0.116052,6,0.0,0.209760,0.187398,1,...,1,5,0.186095,0.509927,0.0,0.351168,0.333333,0.0,0.666667,0.5,0.500,0.333333,2,0.583333,6,0.333333,5,2,0.995025,2,0.75,0.589563,5,5,2,0.224037,0.153565,0.000000,0.0,0.0,0.0,0,0,0,0.0,1.000000,0.50,8,4,0.298709




 test  Table


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20.0,3,80.0,11622.0,1,0,3,3,1,4,0,11,1,2,0,2,5.0,6.0,1961.0,1961.0,1,0,11,3,3,0.0,3,4,1,4,4,4,5,468.0,4,144.0,270.0,882.0,0,4,1,3,896.0,0.0,0.0,896.0,0.0,0.0,1.0,0.0,2.0,1.0,4,5.0,7,0.0,0,2,1961.0,3,1.0,730.0,4,5,2,140.0,0.0,0.0,0.0,120.0,0.0,0,3,0,0.0,6.0,2010.0,9,4
1,1462,20.0,4,81.0,14267.0,1,0,0,3,1,0,0,11,2,2,0,2,6.0,6.0,1958.0,1958.0,3,0,12,4,2,108.0,3,4,1,4,4,4,1,923.0,6,0.0,406.0,1329.0,0,4,1,3,1329.0,0.0,0.0,1329.0,0.0,0.0,1.0,1.0,3.0,1.0,3,6.0,7,0.0,0,2,1958.0,3,1.0,312.0,4,5,2,393.0,36.0,0.0,0.0,0.0,0.0,0,0,1,12500.0,6.0,2010.0,9,4
2,1463,60.0,4,74.0,13830.0,1,0,0,3,1,4,0,8,2,2,0,4,5.0,5.0,1997.0,1998.0,1,0,11,3,3,0.0,3,4,2,3,4,4,3,791.0,6,0.0,137.0,928.0,0,2,1,3,928.0,701.0,0.0,1629.0,0.0,0.0,2.0,1.0,3.0,1.0,4,6.0,7,1.0,5,2,1997.0,1,2.0,482.0,4,5,2,212.0,34.0,0.0,0.0,0.0,0.0,0,3,0,0.0,3.0,2010.0,9,4
3,1464,60.0,4,78.0,9978.0,1,0,0,3,1,4,0,8,2,2,0,4,6.0,6.0,1998.0,1998.0,1,0,11,3,2,20.0,3,4,2,4,4,4,3,602.0,6,0.0,324.0,926.0,0,0,1,3,926.0,678.0,0.0,1604.0,0.0,0.0,2.0,1.0,3.0,1.0,3,7.0,7,1.0,3,2,1998.0,1,2.0,470.0,4,5,2,360.0,36.0,0.0,0.0,0.0,0.0,0,0,0,0.0,6.0,2010.0,9,4
4,1465,120.0,4,43.0,5005.0,1,0,0,1,1,4,0,22,2,2,4,2,8.0,5.0,1992.0,1992.0,1,0,7,11,3,0.0,2,4,2,3,4,4,1,263.0,6,0.0,1017.0,1280.0,0,0,1,3,1280.0,0.0,0.0,1280.0,0.0,0.0,2.0,0.0,2.0,1.0,3,5.0,7,0.0,0,2,1992.0,2,2.0,506.0,4,5,2,0.0,82.0,0.0,0.0,144.0,0.0,0,0,0,0.0,1.0,2010.0,9,4


## Prepare Data:

In [0]:
def preproc(X_train, X_valid, X_test):
    input_list_train = []
    input_list_valid = []
    input_list_test  = []

    # Label Encode Cat cols.
    for c in cat_cols:
        input_list_train.append(X_train[c].values)
        input_list_valid.append(X_valid[c].values)
        input_list_test.append(X_test[c].values)

    # Add the rest of the columns.
    input_list_train.append(X_train[num_cols].values)
    input_list_valid.append(X_valid[num_cols].values)
    input_list_test.append(X_test[num_cols].values)

    return input_list_train, input_list_valid, input_list_test

## Create Model:

In [0]:
# Create the network.
def create_model(data, cat_cols, num_cols):
    inputs, outputs = [], []

    for col in cat_cols:
        n_uniques = int(data[col].nunique())
        embed_dim = int(min(np.ceil((n_uniques)/2), 50))
        inp = layers.Input(shape=(1,))
        out = layers.Embedding(n_uniques + 1, embed_dim, name=col)(inp)
        out = layers.SpatialDropout1D(0.3)(out)
        out = layers.Reshape(target_shape=(embed_dim,))(out)
        inputs.append(inp)
        outputs.append(out)

    input_num = layers.Input(shape=(len(num_cols),))
    x         = layers.Concatenate(axis=1)([*outputs, input_num])
    x         = layers.BatchNormalization()(x)

    # Model Structure.
    x = layers.Dense(300, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Dense(300, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)

    y = layers.Dense(1)(x)

    model = Model(inputs=[*inputs, input_num], outputs=y)
    model.compile(loss='mean_squared_error', 
                  optimizer='adam',
                  metrics=['mae'])
    return model

In [108]:
# Train the network.
X = train[num_cols + cat_cols].reset_index(drop=True)
y = train[target_col].reset_index(drop=True)

X_test  = test[num_cols + cat_cols].reset_index(drop=True)
data    = pd.concat([X, X_test], axis=0).reset_index(drop=True)

shape(X, X_test, y, data)

~> X       has  1,460     rows, and  79   columns.
~> X_test  has  1,459     rows, and  79   columns.
~> y       has  1,460     rows, and  1    columns.
~> data    has  2,919     rows, and  79   columns.


## KFold:

**NOTE**: Remeber to change the metrics and the other things based on your problem type.

In [0]:
oof_preds  = np.zeros((len(X_train)))
test_preds = np.zeros((len(X_test)))

skf = KFold(n_splits=50, shuffle=True, random_state=44)

for train_idx, valid_idx in skf.split(X, y.values):
    # print(len(train_idx), len(valid_idx))
    X_train, X_valid = X.iloc[train_idx, :], X.iloc[valid_idx, :]
    y_train, y_valid = y[train_idx], y[valid_idx]

    X_train, X_valid = X_train.reset_index(drop=True), X_valid.reset_index(drop=True)

    X_train_pre, X_valid_pre, X_test_pre = preproc(X_train, X_valid, X_test)
    
    model = create_model(data, cat_cols, num_cols)
    
    es = callbacks.EarlyStopping(monitor='val_mean_absolute_error',
                                 min_delta=0.001,
                                 patience=5,
                                 verbose=1,
                                 mode='min',
                                 baseline=None,
                                 restore_best_weights=True)
    rlr = callbacks.ReduceLROnPlateau(monitor='val_mean_absolute_error',
                                      factor=0.5,
                                      patience=3,
                                      min_lr=1e-6,
                                      mode='min',
                                      verbose=1)
    model.fit(X_train_pre,
              y_train,
              validation_data=(X_valid_pre, y_valid),
              verbose=1,
              batch_size=16,
              callbacks=[es, rlr],
              epochs=30)
    
    valid_fold_preds = model.predict(X_valid_pre)
    test_fold_preds  = model.predict(X_test_pre)
    # oof_preds[valid_idx] = valid_fold_preds.ravel()
    test_preds += test_fold_preds.ravel()

    print(f'Accuracy: {bg(metrics.mean_absolute_error(y_valid, valid_fold_preds))}', end='\n\n')
    K.clear_session()

Train on 1430 samples, validate on 30 samples
Epoch 1/30
1430/1430 [==============================] - 5s 3ms/step - loss: 1.2873 - mean_absolute_error: 0.9059 - val_loss: 0.1485 - val_mean_absolute_error: 0.2904
Epoch 2/30
1430/1430 [==============================] - 1s 910us/step - loss: 0.6366 - mean_absolute_error: 0.6276 - val_loss: 0.0760 - val_mean_absolute_error: 0.2243
Epoch 3/30
1430/1430 [==============================] - 1s 930us/step - loss: 0.3823 - mean_absolute_error: 0.4972 - val_loss: 0.0550 - val_mean_absolute_error: 0.1802
Epoch 4/30
1430/1430 [==============================] - 1s 904us/step - loss: 0.2400 - mean_absolute_error: 0.3902 - val_loss: 0.0306 - val_mean_absolute_error: 0.1247
Epoch 5/30
1430/1430 [==============================] - 1s 895us/step - loss: 0.1465 - mean_absolute_error: 0.3027 - val_loss: 0.0131 - val_mean_absolute_error: 0.0847
Epoch 6/30
1430/1430 [==============================] - 1s 917us/step - loss: 0.0879 - mean_absolute_error: 0.2338 -